In [50]:
import json
import pandas as pd
from pathlib import Path
from typing import NamedTuple, Tuple


In [24]:
res_patgh = Path("./Results").resolve()

methods = ["original", "stage_1", "stage_2"]

In [56]:
class Results(NamedTuple):
    reported_cost: float
    lines: pd.DataFrame
    assignments: pd.DataFrame
    flows: pd.DataFrame
    mt_cost: float
    mod_cost: float
    mod_cost_total: float
    dropped_requests_count: int

def get_results(path: Path) -> Results:
    main_result = json.load((path / "metrics.json").open())
    reported_cost = main_result["objective_value"]
    lines = pd.read_csv(path / "used_lines.csv")
    assignments = pd.read_csv(path / "passenger_assignments.csv")
    flows = pd.read_csv(path / "flows.csv")
    mt_cost = lines["line_cost"].sum()
    mod_cost = assignments["mod_cost"].sum()
    mod_cost_total = flows['cost'].sum()
    return Results(
        reported_cost=reported_cost,
        lines=lines,
        assignments=assignments,
        flows=flows,
        mt_cost=mt_cost,
        mod_cost=mod_cost,
        mod_cost_total=mod_cost_total,
        dropped_requests_count=len(assignments[assignments['line'] == 'Dropped'])
    )

def load_results_all_methods(base_path: Path) -> Tuple[dict, pd.DataFrame]:
    loaded_results = {}
    for method in methods:
        loaded_results[method] = get_results(base_path / method)

    methods_selected_results = {}
    for method, results in loaded_results.items():
        methods_selected_results[method] = {
            "reported_cost": results.reported_cost,
            "mt_cost": results.mt_cost,
            "mod_cost": results.mod_cost,
            "total_cost_excluding_flow_costs": results.mt_cost + results.mod_cost,
            "mod_cost_empty_trips": results.mod_cost_total - results.mod_cost,
            "mod_flow_cost_total": results.mod_cost_total,
            "dropped_requests_count": results.dropped_requests_count
        }
    # create a dataframe from the dictionary
    results_df = pd.DataFrame.from_dict(methods_selected_results, orient='index')

    return loaded_results, results_df

# 100% Demand

In [49]:
full_exp_path = res_patgh / "100_percent/budget_10000"
original_path = full_exp_path / 'original'
stage_1_path = full_exp_path / 'stage_1'
stage_2_path = full_exp_path / 'stage_2'


original_results = get_results(original_path)
stage_1_results = get_results(stage_1_path)
stage_2_results = get_results(stage_2_path)


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Workspaces\\Cornell\\line_planning\\Results\\100_percent\\budget_10000\\stage_2\\used_lines.csv'

In [21]:
# crete a dictionary
methods = {
    "Original": original_results,
    "Stage 1": stage_1_results,
    "Stage 2": stage_2_results,
}
methods_selected_results = {}
for method, results in methods.items():
    methods_selected_results[method] = {
        "reported_cost": results.reported_cost,
        "mt_cost": results.mt_cost,
        "mod_cost": results.mod_cost,
        "mod_cost_empty_trips": results.mod_cost_total - results.mod_cost,
        "mod_cost_total": results.mod_cost_total,
    }

# create a dataframe from the dictionary
results_df = pd.DataFrame.from_dict(methods_selected_results, orient='index')
results_df

NameError: name 'original_results' is not defined

# 1% Demand, budget 10000

In [58]:
results_1_percent_budget_10000, results_df_1_percent_budget_10000 = load_results_all_methods(
    res_patgh / "1_percent/budget_10000"
)
results_df_1_percent_budget_10000


,reported_cost,mt_cost,mod_cost,total_cost_excluding_flow_costs,mod_cost_empty_trips,mod_flow_cost_total,dropped_requests_count
original,11568.0,9847.0,23601.0,33448.0,84970.0,108571.0,0
stage_1,67.0,644.0,9268.0,9912.0,23274.0,32542.0,63
stage_2,52.0,0.0,7288.0,7288.0,2650.0,9938.0,78


In [37]:
original_assignments = loaded_results['original'].assignments

# 1% Demand, budget 30000

In [74]:
results_1_percent_budget_30000, results_df_1_percent_budget_30000 = load_results_all_methods(
    res_patgh / "1_percent/budget_30000"
)
results_df_1_percent_budget_30000

,reported_cost,mt_cost,mod_cost,total_cost_excluding_flow_costs,mod_cost_empty_trips,mod_flow_cost_total,dropped_requests_count
original,11568.0,9847.0,16738.0,26585.0,6777.0,23515.0,42
stage_1,130.0,5083.0,24913.0,29996.0,10589.0,35502.0,0
stage_2,110.0,1600.0,21181.0,22781.0,6780.0,27961.0,20


In [73]:
results_1_percent_budget_30000_stage_1 = results_1_percent_budget_30000['stage_1']
results_1_percent_budget_30000['stage_1'].flows['flow']

0      1.0
1      1.0
2      3.0
3      4.0
4      1.0
      ... 
375    1.0
376    1.0
377    1.0
378    1.0
379    1.0
Name: flow, Length: 380, dtype: float64

In [69]:
results_1_percent_budget_30000_stage_1.assignments[results_1_percent_budget_30000_stage_1.assignments['line'] == 'no_MT']
52 + (130-52) * 2

208